# Data Cleaning 
The following demonstrates how we processed our data to make it analysis ready.

## Our python imports

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import duckdb

Here we import our title.basics.tsv file in its raw form.

In [2]:

movies = pd.read_csv('title.basics.tsv', delimiter="\t", low_memory=False)

Here we look at  the first 10 rows of our dataset to decide how we are going to clean it.

In [3]:
print(movies.shape)
movies.head(10)

(11169285, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In the following cell we filter out many of the things that are irrilevant to our research question and clean our data by removing empty values marked as "\N" in the original dataset. Since our focus is on movies, we also filter out data related to short films or tv series or other types that the original dataset had. We also limit our analysis to movies that were released in the 21st century so that it is more relevant to our generation. 

Since our dataset is huge we had to do multiple filtering steps and cleaning to get a clean, nice, and relatively smaller dataset that we can easily run our analysis on for later parts of this project. We save the filtered out dataset in a csv file that we titled "cleaned_data" and we will refer to it later when we do more analysis and when we do combined analysis with the movie ratings dataset.


In [4]:
# Remove columns we don't need for our research question:
columns_to_remove = ['isAdult', 'originalTitle', 'endYear']
movies = movies.drop(columns=columns_to_remove)

# Filter out rows where startYear is '\N'
df = movies[movies['startYear'] != '\\N'].copy()

# Convert startYear from string to int
df.loc[:, 'startYear'] = df['startYear'].astype(int)

# Filter out shows
df = df[df['titleType'] == "movie"]

# Filter out years before 2000
df = df[df['startYear'] >= 2000]

# Filter out years after 2024
df = df[df['startYear'] <= 2024]

# Save the cleaned data 
df.to_csv('cleaned_data.csv', index=False)

Here we save our cleaned data to a new DataFrame that we will use in the following steps.

In [5]:
cleaned_movies_df = pd.read_csv('cleaned_data.csv')

Here, we display a summary of our cleaned data. We see that the size of our newly cleaned data is much smaller than the original one with 338244 entries. 

In [6]:
cleaned_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336388 entries, 0 to 336387
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          336388 non-null  object
 1   titleType       336388 non-null  object
 2   primaryTitle    336386 non-null  object
 3   startYear       336388 non-null  int64 
 4   runtimeMinutes  336388 non-null  object
 5   genres          336388 non-null  object
dtypes: int64(1), object(5)
memory usage: 15.4+ MB


Here, we display the first 10 entries of our new cleaned dataset to visualize it and make sure we did our cleaning right:

In [7]:
cleaned_movies_df.head(10)

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres
0,tt0011801,movie,Tötet nicht mehr,2019,\N,"Action,Crime"
1,tt0015414,movie,La tierra de los toros,2000,60,\N
2,tt0035423,movie,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance"
3,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,2020,70,Drama
4,tt0067758,movie,"Simón, contamos contigo",2015,81,"Comedy,Drama"
5,tt0069049,movie,The Other Side of the Wind,2018,122,Drama
6,tt0070596,movie,Socialist Realism,2023,78,Drama
7,tt0077684,movie,Histórias de Combóios em Portugal,2022,46,Documentary
8,tt0082328,movie,Embodiment of Evil,2008,94,Horror
9,tt0088751,movie,The Naked Monster,2005,100,"Comedy,Horror,Sci-Fi"


Here we import another file which have the ratings and the number of votes for each movie.

In [8]:
ratings_df = pd.read_csv("title.ratings.tsv", delimiter = "\t")

Here we combined both tables using SQL to do further analysis that depends on the ratings of each movie as well as the details in previous cleaned_movies_df.

In [9]:

joined_df = duckdb.sql("SELECT * FROM cleaned_movies_df LEFT JOIN ratings_df ON cleaned_movies_df.tconst = ratings_df.tconst").to_df()
joined_df.head(10)

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,tconst_1,averageRating,numVotes
0,tt10843306,movie,The Beach House,2019,88,"Drama,Horror,Mystery",tt10843306,5.3,8730.0
1,tt10843324,movie,Undefiled,2024,115,"Drama,Thriller",tt10843324,5.9,77.0
2,tt10843680,movie,What Lies West,2019,79,Drama,tt10843680,5.6,136.0
3,tt10843712,movie,Five Years North,2020,86,Documentary,tt10843712,8.2,91.0
4,tt10843814,movie,The Candy Witch,2020,94,Horror,tt10843814,3.3,1085.0
5,tt10844026,movie,"All Male, All Nude: Johnsons",2019,83,Documentary,tt10844026,6.6,51.0
6,tt10844140,movie,Fédro,2021,88,Documentary,tt10844140,7.3,20.0
7,tt10844816,movie,Scandalous: The True Story of the National Enq...,2019,97,Documentary,tt10844816,6.4,539.0
8,tt10844900,movie,Qw,2019,10062,Drama,tt10844900,6.9,38.0
9,tt10845262,movie,Miracle in Cell No. 7,2019,120,Drama,tt10845262,7.5,2053.0


Here, we save our cleaned data into csv files: 

In [10]:
df.to_csv('cleaned_data.csv', index=False)
joined_df.to_csv('joined_data.csv', index=False)